In [4]:
!pip install -q rioxarray


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [33]:
import numpy as np
from scipy.ndimage import label
from PIL import Image
import rioxarray
import xarray as xr
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [63]:
from tqdm import tqdm

In [20]:
imagedir = Path('/home/data/kenya/images/')
maskdir = Path('/home/data/kenya/labels/')

In [46]:
buildingdir = Path('/home/data/kenya-binary-masks/buildings/')
cropsdir = Path('/home/data/kenya-binary-masks/crop_fields/')

In [47]:
image_files = os.listdir('/home/data/kenya/images/')
mask_files = os.listdir('/home/data/kenya/labels/')

In [69]:
for mask_file in tqdm(mask_files):
    example_groundtruth = rioxarray.open_rasterio(maskdir / mask_file)
    building_groundtruth = xr.where(example_groundtruth % 2 == 1, 1, 0)
    field_groundtruth = xr.where(example_groundtruth==2, 1, 0)
    
    building_masks, building_num_labels = label(building_groundtruth)
    building_object_masks = []
    for i in range(1, building_num_labels + 1):
        building_object_mask = xr.where(building_masks == i, 1, 0)
        building_object_mask_xr = xr.DataArray(
            building_object_mask, 
            coords=building_groundtruth.coords,
            dims=building_groundtruth.dims,
            attrs=building_groundtruth.attrs)
        building_object_masks.append(building_object_mask_xr)
        
    field_masks, field_num_labels = label(field_groundtruth)
    field_object_masks = []
    for i in range(1, field_num_labels + 1):
        field_object_mask = xr.where(field_masks == i, 1, 0)
        field_object_mask_xr = xr.DataArray(
            field_object_mask, 
            coords=field_groundtruth.coords,
            dims=field_groundtruth.dims,
            attrs=field_groundtruth.attrs)
        field_object_masks.append(field_object_mask_xr)

    for i, building_object_mask in enumerate(building_object_masks):
        building_object_mask.rio.to_raster(buildingdir / (mask_file[:-4]+'_b'+str(i)+'.tif'))

    for i, field_object_mask in enumerate(field_object_masks):
        field_object_mask.rio.to_raster(cropsdir / (mask_file[:-4]+'_f'+str(i)+'.tif'))

100%|███████████████████████████████████████████████████████████████████████████████| 2041/2041 [01:19<00:00, 25.61it/s]
